In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(0)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(0)

# 데이터셋 가져오기
mnist_train = dsets.STL10(root='STL10_data/',
                          split="train", 
                          transform=transforms.ToTensor(), 
                          download=True)

mnist_test = dsets.STL10(root='STL10_data/',
                         split="test",
                         transform=transforms.ToTensor(),
                         download=True)

# 하이퍼 파라미터 설정
learning_rate = 0.001
training_epochs = 40
batch_size = 64

# 데이터 로더 정의
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)


  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting STL10_data/stl10_binary.tar.gz to STL10_data/
Files already downloaded and verified


In [2]:

# CNN 모델 정의
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 96, 96, 3)
        #    Conv     -> (?, 96, 96, 32)
        #    Pool     -> (?, 48, 48, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 48, 48, 32)
        #    Conv      ->(?, 48, 48, 64)
        #    Pool      ->(?, 24, 24, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # 세번째층
        # ImgIn shape=(?, 24, 24, 64)
        #    Conv      ->(?, 24, 24, 128)
        #    Pool      ->(?, 12, 12, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(12 * 12 * 128, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

    
# CNN 모델 생성
model = CNN().to(device)

# 로스와 최적화 함수 설정
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 데이터 로드
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

# 트레이닝 데이터셋으로 epoch 실행
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
        

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

총 배치의 수 : 78
[Epoch:    1] cost = 1.83426309
[Epoch:    2] cost = 1.39173377
[Epoch:    3] cost = 1.17180395
[Epoch:    4] cost = 1.02868271
[Epoch:    5] cost = 0.850036621
[Epoch:    6] cost = 0.699811935
[Epoch:    7] cost = 0.559210241
[Epoch:    8] cost = 0.444498241
[Epoch:    9] cost = 0.31807518
[Epoch:   10] cost = 0.242877632
[Epoch:   11] cost = 0.194770262
[Epoch:   12] cost = 0.0987904817
[Epoch:   13] cost = 0.0872299746
[Epoch:   14] cost = 0.0641077682
[Epoch:   15] cost = 0.0357013643
[Epoch:   16] cost = 0.0434335321
[Epoch:   17] cost = 0.0265754275
[Epoch:   18] cost = 0.0335303508
[Epoch:   19] cost = 0.0713022947
[Epoch:   20] cost = 0.0684004873
[Epoch:   21] cost = 0.0555687919
[Epoch:   22] cost = 0.0607545562
[Epoch:   23] cost = 0.0204086602
[Epoch:   24] cost = 0.00459905528
[Epoch:   25] cost = 0.0014774584
[Epoch:   26] cost = 0.00103421952
[Epoch:   27] cost = 0.000837682863
[Epoch:   28] cost = 0.000705713232
[Epoch:   29] cost = 0.000605451118
[Epoch:  

In [3]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=32)

import numpy as np

accuracy_list = []
with torch.no_grad():
    accuracy = 0
    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        X_test = X.to(device)
        Y_test = Y.to(device)
        # 학습을 진행하지 않을 것이므로 torch.no_grad()

        prediction = model(X_test)
        accuracy += (torch.argmax(prediction, 1) == Y_test).float().mean()
    accuracy /= len(data_loader)
    print('Accuracy:', accuracy)

Accuracy: tensor(0.5803)
